In [1]:
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
import pandas as pd
from sklearn import ensemble
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
def convert_sample(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image,[72,72]).numpy()
    image = image.reshape(1,-1)
    return image

In [3]:
X = np.load('Xtrain.npy')
print(X.shape)
X = np.vstack(list(map(convert_sample,X)))
X = StandardScaler(with_mean=0, with_std=1).fit_transform(X)
print(f'Shape of training data features (observations,features): {X.shape}')

y = np.load('ytrain.npy')
y = y.reshape(-1,)    
print(f'Shape of training data labels (observations,): {y.shape}')

Xtest = np.load('Xtest.npy')
Xtest = np.vstack(list(map(convert_sample,Xtest)))
Xtest = StandardScaler(with_mean=0, with_std=1).fit_transform(Xtest)
print(f'Shape of training data features (observations,features): {Xtest.shape}')

(26214, 96, 96, 3)


c:\ProgramData\anaconda3\envs\amlfall23\lib\site-packages\sklearn\utils\_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


Shape of training data features (observations,features): (26214, 5184)
Shape of training data labels (observations,): (26214,)
Shape of training data features (observations,features): (1638, 5184)


c:\ProgramData\anaconda3\envs\amlfall23\lib\site-packages\sklearn\utils\_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


In [4]:
X_train, X_val, y_train, y_val = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [5]:
subset_train = np.random.choice(X_train.shape[0], 1000, replace=False)
subset_val = np.random.choice(X_val.shape[0], 500, replace=False)

X_train_sub = X_train[subset_train]
y_train_sub = y_train[subset_train]

X_val_sub = X_val[subset_val]
y_val_sub = y_val[subset_val]

subset_trains = np.random.choice(X_train.shape[0], 2000, replace=False)
subset_vals = np.random.choice(X_val.shape[0], 1000, replace=False)

X_train_subs = X_train[subset_trains]
y_train_subs = y_train[subset_trains]

X_val_subs = X_val[subset_vals]
y_val_subs = y_val[subset_vals]

In [6]:
# kernels = ['linear', 'poly', 'rbf'] 
# Cs = [0.1, 1.0, 2.5, 5.0, 10.0, 100.0] 
# Ds = [1,2,3,4,5,6]
# gamma = [0.1, 1, 10]


# results_svm = []

# for kernel in kernels:
#     if kernel == "linear":
#         for C in Cs:
#                 print(f"Training SVM with kernel = {kernel} and C = {C}...")
#                 svm_current = svm.SVC(kernel=kernel, C=C, gamma="scale")
#                 svm_current.fit(X_train_sub, y_train_sub)
#                 y_val_hat = svm_current.predict(X_val_sub)
#                 accuracy = accuracy_score(y_val_hat, y_val_sub)

#                 results_svm.append([accuracy, kernel, C])

#     elif kernel == "rbf": 
#         for C in Cs:
#             for g in gamma: 
#                 print(f"Training SVM with kernel = {kernel}, gamma = {g} and C = {C}...")
#                 svm_current = svm.SVC(kernel=kernel, C=C, gamma=g)
#                 svm_current.fit(X_train_sub, y_train_sub)
#                 y_val_hat = svm_current.predict(X_val_sub)
#                 accuracy = accuracy_score(y_val_hat, y_val_sub)

#                 results_svm.append([accuracy, kernel, C, g])


#     elif kernel == "poly": 
#          for C in Cs: 
#               for d in Ds: 
#                 for g in gamma: 
#                     print(f"Training SVM with kernel = {kernel} degree = {d}, gamma = {g} and C = {C}...")
#                     svm_current = svm.SVC(kernel=kernel, C=C, degree=d, gamma=g)
#                     svm_current.fit(X_train_sub, y_train_sub)
#                     y_val_hat = svm_current.predict(X_val_sub)
#                     accuracy = accuracy_score(y_val_hat, y_val_sub)

#                     results_svm.append([accuracy, kernel, C, d, g])
                    


# results_svm = pd.DataFrame(results_svm)
# results_svm.columns = ['Accuracy', 'Kernel', 'C', "d", "g"]
# print(results_svm)

In [7]:
# results[results['Accuracy'] == results['Accuracy'].max()]

In [8]:
svm_best = svm.SVC(kernel='rbf', C = 1.0)

# Use both training and validation data to fit it (np.concatenate "stacks" the array like rbind in R)
svm_best.fit(np.concatenate([X_train_subs, X_val_subs]), np.concatenate([y_train_subs, y_val_subs]))



SVC()

In [9]:
#Predict on test data
ytest_hat_svm = svm_best.predict(Xtest)



In [10]:
# max_depth = [None, 2, 10, 20]
# n_estimators_list = [50, 100, 300]
# min_samples_split_list = [3, 5, 15, 33]
# min_samples_leaf_list = [3, 5, 15, 33]

# results_rf = []
# for d in max_depth:
#     for n_estimators in n_estimators_list:
#         for min_samples_split in min_samples_split_list:
#             for min_samples_leaf in min_samples_leaf_list:
#                 print(f"Training for n_estimators={n_estimators}, min_samples_split={min_samples_split}, min_samples_leaf={min_samples_leaf}, max_depth={d}")

#                 rf_current = ensemble.RandomForestClassifier(
#                     max_depth = d,
#                     n_estimators=n_estimators,
#                     min_samples_split=min_samples_split,
#                     min_samples_leaf=min_samples_leaf,
#                     )
#                 rf_current.fit(X_train_sub, y_train_sub)
#                 y_val_hat = rf_current.predict(X_val_sub)
#                 accuracy_rf = accuracy_score(y_val_sub, y_val_hat)

#                 results_rf.append([accuracy_rf, n_estimators, min_samples_split, min_samples_leaf, d])

# results_rf = pd.DataFrame(results_rf)
# results_rf.columns = ['accuracy_rf', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'max_depth']
# print(results_rf)
# results_rf[results_rf['accuracy_rf'] == results_rf['accuracy_rf'].max()]

In [11]:
rf_best = ensemble.RandomForestClassifier(
    max_depth = None, 
    n_estimators = 100, 
    min_samples_split = 33,
    min_samples_leaf = 5
)

rf_best.fit(np.concatenate([X_train_subs, X_val_subs]), np.concatenate([y_train_subs, y_val_subs]))

ytest_hat_rf = rf_best.predict(Xtest)

In [12]:
# from sklearn import ensemble

# n_estimators_list = [100, 500, 1000, 1500]
# min_samples_split_list = [2, 10, 30]
# min_samples_leaf_list = [2, 10, 30]
# learning_rate_list = [0.01, 0.10, 0.50]

# results_gb = []

# for n_estimators in n_estimators_list:
#     for min_samples_split in min_samples_split_list:
#         for min_samples_leaf in min_samples_leaf_list:
#             for learning_rate in learning_rate_list:
#                 print(f"Training with n_estimators={n_estimators}, min_samples_split={min_samples_split}, min_samples_leaf={min_samples_leaf}, learning_rate={learning_rate}...")

#                 gbt_current = ensemble.GradientBoostingClassifier(n_estimators=n_estimators, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, learning_rate=learning_rate)
#                 gbt_current.fit(X_train_sub, y_train_sub)
#                 y_val_hat = gbt_current.predict(X_val_sub)
#                 acc = accuracy_score(y_val_sub, y_val_hat)

#                 results_gb.append([acc, n_estimators, min_samples_split, min_samples_leaf, learning_rate])

# results_gb = pd.DataFrame(results_gb)
# results_gb.columns = ['Accuracy_gb', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'learning_rate']
# results_gb[results_gb['Accuracy_gb'] == results_gb['Accuracy_gb'].max()]

In [13]:
# param_dist = {
#     'n_estimators': [50, 100, 200, 300],
#     'min_samples_split': [2, 10, 25],
#     'min_samples_leaf': [2, 10, 25],
#     'learning_rate': [0.01, 0.10, 0.50],
#     'max_depth': [None, 2, 5, 15]
# }

# gbt = GradientBoostingClassifier()

# # Use 3-fold cross-validation and run 20 iterations of randomized search
# random_search = RandomizedSearchCV(gbt, param_distributions=param_dist, n_iter=15, scoring='accuracy', cv=3, n_jobs=-1, verbose=3)

# random_search.fit(X_train_sub, y_train_sub)

# print("Best parameters found: ", random_search.best_params_)
# print("Highest accuracy found: ", random_search.best_score_)


In [14]:
gb_best = GradientBoostingClassifier(n_estimators = 300, min_samples_split=10, min_samples_leaf=2, max_depth=2, learning_rate=0.2)
gb_best.fit(np.concatenate([X_train_subs, X_val_subs]), np.concatenate([y_train_subs, y_val_subs]))

ytest_hat_gb = gb_best.predict(Xtest)

In [15]:
y_test_hat_combined = np.c_[ytest_hat_gb, ytest_hat_rf, ytest_hat_svm]
y_test_hat_combined = np.round(np.sum(y_test_hat_combined, axis=1) / y_test_hat_combined.shape[1]).astype(int)

In [16]:
y_test_hat_combined= pd.DataFrame({
    'Id': list(range(len(y_test_hat_combined))),
    'Predicted': y_test_hat_combined.reshape(-1,),
})
y_test_hat_combined.to_csv("ytest_hat.csv", index=False)

In [17]:
y_test_hat_combined.shape


(1638, 2)